# Taller de supervisado de enfermedades coronarias en Sudáfrica

El objetivo de este taller es tener un acercamiento a los modelos de clasificación de regresión logística y K-NN, utilizando como caso de estudio una población sudafricana con alto riesgo de enfermedad coronaria. El dataset, que en cada registro presenta un individuo hombre, incluye las siguientes variables:

- sbp: presión sanguínea sistólica
- tobacco: cantidad de tabaco acumulada (en kg)
- ldl: colesterol de lipoproteinas de baja densidad
- adiposity: grado de tejido adiposo
- famhist: indica si la familia del individuo tiene historia de enfermedades coronarias (“Present”) o no (“Absent”)
- typea: comportamiento tipo A
- obesidad: grado de obesidad
- alcohol: consumo de alcohol actual
- age: edad
- chd: variable objetivo que indica si el individuo tiene enfermedad coronaria sí (1) o no (0)

# 0. Librerías a importar

In [1]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors, metrics
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# 1. Cargar, explorar y preparar el dataset

In [2]:
corazon = pd.read_csv('SAheart.csv')
corazon.head()

FileNotFoundError: [Errno 2] File SAheart.csv does not exist: 'SAheart.csv'

In [ ]:
corazon.info()

In [ ]:
corazon.describe(include='all')

In [ ]:
corazon.quantile(np.arange(0,1,0.1))

In [ ]:
corazon.describe(include=['O'])

In [ ]:
corazon['chd'].value_counts()

In [ ]:
tab = corazon['chd'].value_counts()
tab/tab.sum()

### PREGUNTA: 

¿Qué ven de particular en los datos?


Por un lado encontramos que la variable objetivo no está completamente balanceada (65% vs 35% de proporciones para las dos categorías). La diferencia no es, en todo caso, grave, pues hay una gran proporción de la clase minoritaria


In [ ]:
corazon

In [ ]:
print(f"Min ldl: {corazon['ldl'].min()}, Max ldl: {corazon['ldl'].max()}")
print(f"Min sbp: {corazon['sbp'].min()}, Max sbp: {corazon['sbp'].max()}")

También vemos que las escalas de las variables numéricas son bien diferentes, con variables como ldl que van de 0.98 a 15.3 y otras como sbp que van de 101 a 218. Esto implica que, con modelos sensibles a las escalas como K-NN, es necesario reescalar los valores de las variables, para que todas tengan la misma importancia en los modelos finales. En el caso de la regresión logística, el modelo no es sensible a la escala, pero los coeficientes de las variables predictivas no podrán ser comparados directamente para entender las variables mas sensibles en la predicción de la variable dependiente.

Vamos a estandarizar los datos para poder aplicar el modelo K-nn sin problemas, y poder interpretar más fácilmente los coeficientes del modelo de regresión logística.

In [ ]:
vars = corazon.columns
numericVars = corazon.select_dtypes(include='number')
categoricalVars = corazon.select_dtypes(include='object')
numericVars[numericVars.columns] = StandardScaler().fit_transform(numericVars)
corazonStd = pd.concat([numericVars,categoricalVars], axis=1)
corazonStd.describe()

Vamos ahora a explorar las distribuciones de los valores de las variables numéricas del dataset a partir de un diagrama de boxplot:

In [ ]:
corazonStd[numericVars.columns].boxplot(figsize=(15,10))

Encontramos que hay valores excepcionales pero no consideramos que sean anómalos, por lo que no eliminamos registros.

Podemos tratar de entender la influencia de cada variable predictiva en la predicción de la variable objetivo, de manera aislada e independiente de las otras variables, a partir de un plot de densidad. Visualicemos la distribución de los valores de la variable age para los individuos con y sin problemas cardiacos:

In [ ]:
sns.displot(corazon,x='age',hue='chd',kind='kde',fill=True)

En el gráfico anterior podemos ver que las personas de mayor edad son más propensas a desarrollar enfermedades cardíacas

Veamos como se comporta la variable tobacco:

In [ ]:
sns.displot(corazon,x='tobacco',hue='chd',kind='kde',fill=True)

Vemos que las personas que consumen muy poco tabaco son muchísimo menos propensas a desarrollar enfermedades cardiacas, y vicecersa.

# 2. Modelo de regresión logística

Vale la pena recalcar el hecho de que no vamos a seguir aún ningún protocolo de evaluación que permita obtener una idea clara de las capacidades de generalización del modelo a datos diferentes a los del dataset de entrenamiento: vamos a entrenar y a evaluar el modelo sobre el mismo dataset.

Utilizaremos el LogisticRegression del paquete sklearn. Algunos de los parámetros del método son los siguientes:
- penalty: término de fenalización para realizar regularización.
- fit_intercept: para especificar si una constante (el intercepto o sesgo) se debe añadir a la función
- random_state: establece la semilla al generador aleatorio de tal manera que los resultados sean deterministas (se puedan replicar)
- solver: algoritmo a utilizarse en el problema de optimización

Vamos a entrenar un primer modelo de regresión logística sobre el dataset, utilizando solamente la variable ldl, sobre el dataset sin estandarizar:

Para ello haremos uso de la función fit a la cual le pasaremos una matriz de forma (n,m), donde n es la cantidad de muestras y m la cantidad de predictores, y un vector de etiquetas de forma (n,).

Además, podemos ver los coeficientes asociados a cada variable predictiva. En este caso hay una sola variable predictiva, ldl, que presenta un coeficiente de 0.2739.


In [ ]:
X1 = pd.DataFrame(corazon['ldl'])
y = corazon['chd']
logReg1 = LogisticRegression(random_state=0).fit(X1,y)
logReg1.coef_[0][0]

Para poder realizar las predicciones se utiliza el método predict. Se recibe como parámetro la matriz de variables independientes, aunque en este caso tan solo involucra la variable independiente ldl.

Se pueden observar las primeras 5 predicciones.

In [ ]:
preds1 = logReg1.predict(X1)
preds1[0:5]

Con el método predict_proba obtenemos la probabilidad de pertenecer a cada categoría de la clase. En este caso particular podemos observar la probabilidad de que el resultado sea Si

In [ ]:
preds1_prob = logReg1.predict_proba(X1)
preds1_prob[0:5,1]

Con el método score obtenemos la exactitud o accuracy de nuestro modelo, en este caso el 66.6%

In [ ]:
logReg1.score(X1,y)

En este caso, en vez de utilizar la variable ldl para implementar nuestro modelo, lo hacemos con age

In [ ]:
X2 = pd.DataFrame(corazon['age'])
logReg2 = LogisticRegression(random_state=0).fit(X2,y)
preds2 = logReg2.predict(X2)
preds2[0:5]

In [ ]:
preds2_prob = logReg2.predict_proba(X2)
preds2_prob[0:5,1]

Se tiene un mejor ajuste de acuerdo con el nivel de accuracy 67.96%

In [ ]:
logReg2.score(X2,y)

Utilizamos en este caso todas las variables numéricas para implementar el modelo.

In [ ]:
X3 = pd.DataFrame(corazon[numericVars.columns])
y = corazon['chd']
logReg3 = LogisticRegression(random_state=0).fit(X3,y)
preds3 = logReg3.predict(X3)
preds3[0:5]

In [ ]:
preds3_prob = logReg3.predict_proba(X3)
preds3_prob[0:5,1]

Se obtiene un accuracy del 72.29%

In [ ]:
logReg3.score(X3,y)

Utilizamos el conjunto de datos estandarizado con todas las variables numéricas

In [ ]:
X4 = pd.DataFrame(corazonStd[numericVars.columns])
y = corazon['chd']
logReg4 = LogisticRegression(random_state=0).fit(X4,y)
preds4 = logReg4.predict(X4)
preds4[0:5]

In [ ]:
preds4_prob = logReg4.predict_proba(X4)
preds4_prob[0:5,1]

El nivel de exactitud es un poco mejor con un 72.51%

In [ ]:
logReg4.score(X4,y)

# 3. Modelo KNN

Vamos a utilizar ahora el KNeighborsClassifier de sklearn. Algunos parámetros de este método son:

- n_neighbors: la cantidad de vecinos a utilizar para realizar la predicción
- algorithm: algoritmo utilizado para computar los vecinos más cercanos
- metric: métrica para establecer la distancia entre las observaciones

Vamos a entrenar un primer modelo de KNN sobre el dataset sin estandarizar. Hay que tener en cuenta que la variable famhist es categórica con dos variables, por lo que no la vamos a tener en cuenta.

Vamos a considerar las primeras 362 instancias como set de entrenamieno y las últimas como set de test. Cuidado: esto se puede hacer cuando se sabe que los datos no están ordenados. Separamos las variables predictivas de la objetivo.

In [ ]:
corazon.shape
corazon.columns

In [ ]:
train_X = corazon.iloc[0:362, [0,1,2,3,5,6,7,8]]
train_y = corazon.iloc[0:362, 9]

In [ ]:
test_X = corazon.iloc[362:, [0,1,2,3,5,6,7,8]]
test_y = corazon.iloc[362:, 9]

Vamos ahora a predecir la clase de las instancias del test set, utilizando el modelo knn con k=5 y a compararlas con las clases reales, usando un valor de K=5

In [ ]:
k = 5 
knn = neighbors.KNeighborsClassifier(n_neighbors=k) 
knn.fit(train_X,train_y) 
preds_y = knn.predict(test_X)
coincidencias = 0
for i in range(len(preds_y)):
    if(preds_y[i] ==  test_y.values[i]):
        coincidencias+=1
print(f"Con k={k} se encontraron {coincidencias} buenas predicciones")

In [ ]:
k = 15 
knn = neighbors.KNeighborsClassifier(n_neighbors=k) 
knn.fit(train_X,train_y) 
preds_y = knn.predict(test_X)
coincidencias = 0
for i in range(len(preds_y)):
    if(preds_y[i] ==  test_y.values[i]):
        coincidencias+=1
print(f"Con k={k} se encontraron {coincidencias} buenas predicciones")

Vemos que nos va mejor con el modelo que utiliza un parámetro K con valor de 15.